In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import os

In [3]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コード : {finish_pref}')


def land_price_estimate(damage_df):
    value_land = 48100
    damage_df['land_price'] = damage_df['AREA'] * value_land
    return damage_df


def subsidy(damage_df):
    #補助金とそれに対応する移転希望者の割合の辞書を作成
    #0.25~0.26で最小値がある。
    subsidy_dic = {}
    transfer_ratio = 0.652

    # 総和を保存するための空のデータフレームを作成
    subsidy_df = pd.DataFrame(columns=['subsidy_ratio', 'transfer_ratio', 'house_measures_cost', 'house_damage_measures_cost', 'land_measures_cost', 'total_measures_cost', 'without_measures_cost', 'difference'])

    #補助金毎にループ
    for col in subsidy_list:
        #damage_dfにdamage_priceに対する補助する金額の列を追加
        damage_df[f'house_{col * 100}%'] = damage_df['house_price']*col
        #damage_dfにland_priceに対する補助する金額の列を追加
        damage_df[f'land_{col * 100}%'] = damage_df['land_price']*col

        
        # データフレームから指定された割合で行をランダムに選ぶ
        sample_df = damage_df.sample(frac=transfer_ratio)
        
        
        
        #施策なしの損害額
        without_measures_cost = damage_df['damage_price'].sum()
        #施策有の損害額
        house_damage_measures_cost = without_measures_cost - sample_df['damage_price'].sum()
        # 選択された行に含まれる数値の総和を計算 = 補助金
        house_measures_cost = sample_df[f'house_{col * 100}%'].sum()
        #土地の取得費用
        land_measures_cost = sample_df[f'land_{col * 100}%'].sum()
        #施策有の総費用
        total_measures_cost = house_damage_measures_cost + house_measures_cost + land_measures_cost
        #施策有と無しの差
        difference = total_measures_cost - without_measures_cost

        
        
        # 総和を一時的なデータフレームに保存
        temp_df = pd.DataFrame({
            'subsidy_ratio': [f"subsidy_{col * 100}%"],
            'transfer_ratio': [f"transfer_{transfer_ratio * 100}%"],
            'house_measures_cost': [house_measures_cost],
            'house_damage_measures_cost':[house_damage_measures_cost],
            'land_measures_cost': [land_measures_cost],
            'total_measures_cost':[total_measures_cost], 
            'without_measures_cost':[without_measures_cost],
            'difference':[difference]
        })

        
        
        # 結果をメインのデータフレームに追加
        subsidy_df = pd.concat([subsidy_df, temp_df], ignore_index=True)
        
        
        
        if difference < 0:
            print(f'移転の最適な補助割合は{col}')
            break
        

    return subsidy_df

    

%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
n = int(start_pref)
while n <= int(finish_pref):
    #岡山県の行政地域ポリゴン
    okayama_polygon_path = './QGIS/岡山県行政地域/N03-23_33_230101.shp'
    gdf_okayama_polygon = gpd.read_file(okayama_polygon_path, encoding='shift-jis')
    #対象地域を倉敷市に絞る
    gdf_kurashiki_polygon = gdf_okayama_polygon[gdf_okayama_polygon['N03_004'] == '総社市']
    
    
    #座標系を変更
    src_proj = 6668 # 変換前の座標系を指定
    dst_proj = 4326  # 変換後の座標系を指定

    gdf_kurashiki_polygon.crs = f'epsg:{src_proj}'  # 変換前座標を指定
    gdf_kurashiki_polygon = gdf_kurashiki_polygon.to_crs(epsg=dst_proj)  # 変換後座標に変換
    
    
    
    #岡山県全体の住宅ポイントデータ
    damage_path = f'./dev/損害額/{n}_step8_1_output.csv'
    damage_df = pd.read_csv(damage_path, encoding='shift-jis')
    
    # geometryを追加
    damage_df['geometry'] = list(zip(damage_df.x, damage_df.y))
    # 座標列をPoint型に変換
    damage_df['geometry'] = damage_df['geometry'].apply(Point)
    
    # pandasのデータフレームをジオデータフレームに変換
    damage_df = gpd.GeoDataFrame(damage_df, geometry='geometry', crs="EPSG:4326")
    
    
    
    #住宅ポイントデータを絞る
    gdf_kurashiki_house = gpd.sjoin(damage_df, gdf_kurashiki_polygon, op='within')
    damage_path = f'./dev/損害額/{n}_step8_1_output.csv'
    damage_df = pd.read_csv(damage_path, encoding='shift-jis')
    
    #土地費用を追加
    damage_df = land_price_estimate(damage_df)
    
    subsidy_df = subsidy(damage_df)
    
    n += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 33
終了する都道府県コードを入力してください
終了する都道府県コード : 33
G:\マイドライブ\akiyamalab\卒論\負担率総集編


c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\imai_14\AppData\Local\Temp\ipykernel_26912\1217558193.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  damage_df[f'land_{col * 100}%'] = damage_df['land_price']*col
C:\Users\imai_14\AppData\Local\Temp\ipykernel_26912\1217558193.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-frag

移転の最適な補助割合は0.21799999999999997


In [4]:
subsidy_df

,subsidy_ratio,transfer_ratio,house_measures_cost,house_damage_measures_cost,land_measures_cost,total_measures_cost,without_measures_cost,difference
0,subsidy_30.0%,transfer_65.2%,1.718532e+12,7.939306e+11,3.172814e+11,2.829744e+12,2.273164e+12,5.565801e+11
1,subsidy_29.9%,transfer_65.2%,1.718302e+12,7.906336e+11,3.172063e+11,2.826142e+12,2.273164e+12,5.529784e+11
2,subsidy_29.799999999999997%,transfer_65.2%,1.709635e+12,7.928181e+11,3.157032e+11,2.818156e+12,2.273164e+12,5.449921e+11
3,subsidy_29.7%,transfer_65.2%,1.703476e+12,7.921872e+11,3.143513e+11,2.810014e+12,2.273164e+12,5.368504e+11
4,subsidy_29.599999999999998%,transfer_65.2%,1.697506e+12,7.909350e+11,3.135612e+11,2.802002e+12,2.273164e+12,5.288382e+11
...,...,...,...,...,...,...,...,...
78,subsidy_22.199999999999996%,transfer_65.2%,1.272416e+12,7.909146e+11,2.350041e+11,2.298335e+12,2.273164e+12,2.517124e+10
79,subsidy_22.099999999999998%,transfer_65.2%,1.267653e+12,7.900523e+11,2.340074e+11,2.291713e+12,2.273164e+12,1.854895e+10
80,subsidy_21.999999999999996%,transfer_65.2%,1.262493e+12,7.895940e+11,2.330559e+11,2.285143e+12,2.273164e+12,1.197893e+10
81,subsidy_21.9%,transfer_65.2%,1.256187e+12,7.888747e+11,2.320218e+11,2.277084e+12,2.273164e+12,3.920046e+09
